# ModSSC | CLI tour

A quick pass through the main CLI entry points and subcommands.

## Objective
- Show the minimal steps to run this component in a notebook setting.
- Provide the exact objects to look at (outputs, shapes, metrics) to confirm it worked.

## Prerequisites
- Python 3.11+.
- `pip install modssc`.
- Optional dependencies depend on datasets and backends. If an import fails, install the matching extra and rerun.

## Outline
1) Imports and configuration
2) Core run (the part that does the work)
3) Sanity checks and outputs



## Notebook notes

This notebook walks through the main CLI entry points and subcommands.


## Helpers


## Imports and configuration



In [ ]:
import shutil
import subprocess
import sys


def run_module(*args):
    cmd = [sys.executable, "-m", "modssc", *args]
    res = subprocess.run(cmd, text=True, capture_output=True)
    return res.returncode, res.stdout.strip(), res.stderr.strip()


def run_entrypoint(*args):
    exe = shutil.which("modssc")
    if exe is None:
        return None
    res = subprocess.run([exe, *args], text=True, capture_output=True)
    return res.returncode, res.stdout.strip(), res.stderr.strip()

## Entry points


In [ ]:
print(run_module("--version"))
entrypoint = run_entrypoint("--version")
print(entrypoint if entrypoint is not None else "modssc entry point not found")

## Datasets CLI


In [ ]:
print(run_module("datasets", "list"))
print(run_module("datasets", "cache", "ls"))

## Sampling CLI


In [ ]:
import json
import tempfile
from pathlib import Path

work_dir = Path(tempfile.mkdtemp(prefix="modssc_cli_"))
plan_path = work_dir / "plan.json"
out_dir = work_dir / "split"

plan = {
    "split": {"kind": "holdout", "test_fraction": 0.2, "val_fraction": 0.1, "stratify": True},
    "labeling": {"mode": "fraction", "value": 0.1, "per_class": True, "min_per_class": 1},
    "imbalance": {"kind": "none"},
    "policy": {"respect_official_test": True},
}

plan_path.write_text(json.dumps(plan), encoding="utf-8")

print(
    run_module(
        "sampling",
        "create",
        "--dataset",
        "toy",
        "--plan",
        str(plan_path),
        "--seed",
        "0",
        "--out",
        str(out_dir),
        "--overwrite",
    )
)
print(run_module("sampling", "show", str(out_dir)))
print(run_module("sampling", "validate", str(out_dir), "--dataset", "toy"))

## Preprocess CLI


In [ ]:
print(run_module("preprocess", "steps", "list"))
print(run_module("preprocess", "models", "list"))

## Graph CLI


In [ ]:
print(run_module("graph", "cache", "ls"))
print(run_module("graph", "views", "cache-ls"))

## Inductive CLI


In [ ]:
print(run_module("inductive", "methods", "list"))

## Transductive CLI


In [ ]:
print(run_module("transductive", "methods", "list"))
print(run_module("transductive", "methods", "info", "nodeformer"))

## Outputs

- The last cells should print key shapes and a minimal metric or artifact summary.
- If something fails early, the error should point to a missing optional dependency.


## Next steps
- Explore the adjacent notebooks in this folder for the other pipeline components.
- If you hit an optional dependency error, install the suggested extra and rerun.
